In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
!pip install pytorch_lightning
!pip install torchtext
!pip install matplotlib
!pip install torchvision
!pip install torch
!pip install tqdm
!pip install timm
!pip install scikit_learn
!pip install pandas
!pip install opencv_contrib_python
!pip install opencv_python_headless
! pip install torchsummary
! pip install einops
!pip install wandb

In [ ]:
import torch
from torch import nn
import torchvision
from torchsummary import summary
from torchvision.models.feature_extraction import create_feature_extractor
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import cv2
from albumentations.pytorch import ToTensorV2
import albumentations as A

import timm
from pytorch_lightning.utilities.model_summary import ModelSummary, summarize
from pytorch_lightning.tuner.tuning import Tuner
from einops.layers.torch import Rearrange
from torchmetrics import ConfusionMatrix
from scipy.ndimage import gaussian_filter1d, convolve1d
from scipy.signal.windows import triang

from torchvision import datasets, transforms, models
from PIL import Image

from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Optional
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar

from scipy.io import loadmat
from collections import OrderedDict

import os
from os import listdir
from os.path import isfile, join
import shutil

import seaborn as sns
%matplotlib inline

from IPython.display import display


In [ ]:

class QualityDataset(Dataset):

    def __init__(self, parent_dir, annotated_path, is_train=False):
        super().__init__()

        self.parent_dir = parent_dir
        self.annotated_file = pd.read_csv(annotated_path)

        if is_train:
            self.transforms = A.Compose([
                A.Resize(384, 512),
                # A.HorizontalFlip(p=0.3),
                A.Normalize(
                    mean=(0.5, 0.5, 0.5),
                    std=(0.5, 0.5, 0.5),
                ),
                ToTensorV2()
            ])
        else:
            self.transforms = A.Compose([
                A.Resize(384, 512),
                A.Normalize(
                    mean=(0.5, 0.5, 0.5),
                    std=(0.5, 0.5, 0.5),
                ),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.annotated_file)

    def __getitem__(self, index):
        info = self.annotated_file.iloc[index]

        image_path = os.path.join(
            self.parent_dir, info["image_name"]
        )
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)["image"]

        rating = info["MOS"]

        rating = torch.tensor(rating, dtype=torch.float32)

        return image, rating

class QualityDataModule(pl.LightningDataModule):

    def __init__(self, parent_dir, train_csv, val_csv, batch_size=128):
        super().__init__()

        self.parent_dir = parent_dir
        self.train_csv = train_csv
        self.val_csv = val_csv
        self.batch_size = batch_size

    def setup(self, stage):
        self.train_data = QualityDataset(self.parent_dir, self.train_csv, is_train=True)
        self.val_data = QualityDataset(self.parent_dir, self.val_csv)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=12)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=12)

# class QualityDataset(Dataset):

#     def __init__(self, parent_dir, annotated_path, is_train=False):
#         super().__init__()

#         self.parent_dir = parent_dir
#         self.annotated_file = pd.read_csv(annotated_path)

#         if is_train:
#             self.transforms = A.Compose([
#                 A.Resize(224, 224),
#                 A.HorizontalFlip(p=0.5),
#                 A.Normalize(
#                     mean=(0.485, 0.456, 0.406),
#                     std=(0.229, 0.224, 0.225),
#                 ),
#                 ToTensorV2()
#             ])
#         else:
#             self.transforms = A.Compose([
#                 A.Resize(224, 224),
#                 A.Normalize(
#                     mean=(0.485, 0.456, 0.406),
#                     std=(0.229, 0.224, 0.225),
#                 ),
#                 ToTensorV2()
#             ])

#     def __len__(self):
#         return len(self.annotated_file)

#     def __getitem__(self, index):
#         info = self.annotated_file.iloc[index]

#         image_path = os.path.join(
#             self.parent_dir, info["image_name"]
#         )
#         image = cv2.imread(image_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = self.transforms(image=image)["image"]

#         rating = info["cate"]

#         rating = torch.tensor(rating, dtype=torch.long)

#         return image, rating

# class QualityDataModule(pl.LightningDataModule):

#     def __init__(self, parent_dir, train_csv, val_csv, batch_size=128):
#         super().__init__()

#         self.parent_dir = parent_dir
#         self.train_csv = train_csv
#         self.val_csv = val_csv
#         self.batch_size = batch_size

#     def setup(self, stage):
#         self.train_data = QualityDataset(self.parent_dir, self.train_csv, is_train=True)
#         self.val_data = QualityDataset(self.parent_dir, self.val_csv)

#     def train_dataloader(self):
#         return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=12)

#     def val_dataloader(self):
#         return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=12)

class ReviewDataset(Dataset):

    def __init__(self, annotated_path, is_train=False):
        super().__init__()

        self.annotated_file = pd.read_csv(annotated_path)

        if is_train:
            self.transforms = A.Compose([
                A.Resize(224, 224),
#                 A.Flip(p=0.5),
                A.Normalize(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5],
                ),
                ToTensorV2()
            ])
        else:
            self.transforms = A.Compose([
                A.Resize(224, 224),
                A.Normalize(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5],
                ),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.annotated_file)

    def __getitem__(self, index):
        info = self.annotated_file.iloc[index]

        image_path = info["Image"]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)["image"]

        rating = info["Rating"]

        rating = torch.tensor(rating, dtype=torch.float32)

        return image, rating

class ReviewDataModule(pl.LightningDataModule):

    def __init__(self, train_csv, val_csv, batch_size=128):
        super().__init__()

        self.train_csv = train_csv
        self.val_csv = val_csv
        self.batch_size = batch_size

    def setup(self, stage):
        self.train_data = ReviewDataset(self.train_csv, is_train=True)
        self.val_data = ReviewDataset(self.val_csv)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=12)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=12)


#LOSS

In [ ]:
import torch
import torch.nn as nn

class RFCLoss(nn.Module):
    def __init__(self, alpha=10, min_delta=0.47, loss=nn.L1Loss(reduction = 'none')):
        super().__init__()

        self.loss = loss
        self.alpha = alpha
        self.min_delta = min_delta

    def forward(self, y_pred, y_true):
        mae_loss = torch.abs(y_pred - y_true)
        weights = 1/(1 + torch.exp(self.alpha * (self.min_delta - mae_loss)))
        loss = weights * self.loss(y_pred, y_true)

        return torch.mean(loss).cuda()

#DPD BLOCK

In [ ]:
from timm.models.layers import DropPath
class DPDBlockV1(nn.Module):
    def __init__(
        self,
        in_channels,
        expand_channels,
        freeze_channels,
        is_downsize=False
    ):
        super().__init__()

        self.in_channels = in_channels
        self.expand_channels = self._make_divisible(expand_channels, in_channels)
        self.freeze_channels = freeze_channels
        self.is_downsize = is_downsize

        self.depthwise_expand = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.in_channels,
                out_channels=self.expand_channels,
                kernel_size=3,
                stride=2 if self.is_downsize else 1,
                padding=1,
                groups=self.in_channels,
                bias=False
            ),
            nn.BatchNorm2d(self.expand_channels),
            nn.ReLU()
        )

        self.pointwise = nn.Sequential(
            CABlock(in_channels=self.expand_channels, ratio=8),
            nn.Conv2d(
                in_channels=self.expand_channels,
                out_channels=self.freeze_channels,
                kernel_size=1,
                stride=1,
                bias=False
            ),
            nn.BatchNorm2d(self.freeze_channels),
            nn.ReLU()
        )

        self.depthwise_freeze = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.freeze_channels,
                out_channels=self.freeze_channels,
                kernel_size=3,
                stride=2 if self.is_downsize else 1,
                padding=1,
                groups=self.freeze_channels,
                bias=False
            ),
            nn.BatchNorm2d(self.freeze_channels),
            nn.ReLU()
        )

        self.skip_att = SABlock()

        self._initialize_weights()

    def forward(self, x):
        inputs = x
        x = self.depthwise_expand(x)
        x = self.pointwise(x)
        x = self.depthwise_freeze(x)

        if not(self.is_downsize) and x.shape == inputs.shape:
            inputs = self.skip_att(inputs)
            x = inputs + x

        return x

    def _make_divisible(self, out_channels, groups):
        ratio = out_channels // groups
        return int(groups * ratio)

    def _initialize_weights(self):
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

class DPDBlockV3(nn.Module):
    def __init__(
        self,
        in_channels,
        expand_channels,
        freeze_channels,
        is_downsize=False
    ):
        super().__init__()

        self.in_channels = in_channels
        self.expand_channels = self._make_divisible(expand_channels, in_channels)
        self.freeze_channels = freeze_channels
        self.is_downsize = is_downsize

        self.depthwise_expand = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.in_channels,
                out_channels=self.expand_channels,
                kernel_size=3,
                stride=2 if self.is_downsize else 1,
                padding=1,
                groups=self.in_channels,
                bias=True
            ),
            nn.BatchNorm2d(self.expand_channels),
            nn.ReLU()
        )

        self.pointwise = nn.Sequential(
            LCABlock(in_channels=self.expand_channels),
            SeperableGhostModule(
                in_channels=self.expand_channels,
                out_channels=self.freeze_channels,
                kernel_size=1,
                stride=1,
                bias=True
            )
        )

        self.depthwise_freeze = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.freeze_channels,
                out_channels=self.freeze_channels,
                kernel_size=3,
                stride=1,
                padding=1,
                groups=self.freeze_channels,
                bias=True
            ),
            nn.BatchNorm2d(self.freeze_channels),
        )

        self.drop_path = DropPath(0.2)

        self._initialize_weights()

    def forward(self, x):
        inputs = x
        x = self.depthwise_expand(x)
        x = self.pointwise(x)
        x = self.depthwise_freeze(x)

        if not(self.is_downsize) and x.shape == inputs.shape:
            x = x + self.drop_path(inputs)

        return x

    def _make_divisible(self, out_channels, groups):
        ratio = out_channels // groups
        return int(groups * ratio)

    def _initialize_weights(self):
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)


class DPDBlockV2(nn.Module):
    def __init__(
        self,
        in_channels,
        expand_channels,
        freeze_channels,
        is_downsize=False
    ):
        super().__init__()

        self.in_channels = in_channels
        self.expand_channels = self._make_divisible(expand_channels, in_channels)
        self.freeze_channels = freeze_channels
        self.is_downsize = is_downsize

        self.depthwise_expand = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.in_channels,
                out_channels=self.expand_channels,
                kernel_size=3,
                stride=2 if self.is_downsize else 1,
                padding=1,
                groups=self.in_channels,
                bias=False
            ),
            nn.BatchNorm2d(self.expand_channels),
            nn.ReLU()
        )

        self.pointwise = nn.Sequential(
            LCABlock(in_channels=self.expand_channels),
            nn.Conv2d(
                in_channels=self.expand_channels,
                out_channels=self.freeze_channels,
                kernel_size=1,
                stride=1,
                bias=False
            ),
            nn.BatchNorm2d(self.freeze_channels),
            nn.ReLU()
        )

        self.depthwise_freeze = nn.Sequential(
            SABlock(),
            nn.Conv2d(
                in_channels=self.freeze_channels,
                out_channels=self.freeze_channels,
                kernel_size=3,
                stride=2 if self.is_downsize else 1,
                padding=1,
                groups=self.freeze_channels,
                bias=False
            ),
            nn.BatchNorm2d(self.freeze_channels),
        )

        self.drop_path = DropPath(0.2)

        self._initialize_weights()

    def forward(self, x):
        inputs = x
        x = self.depthwise_expand(x)
        x = self.pointwise(x)
        x = self.depthwise_freeze(x)

        if not(self.is_downsize) and x.shape == inputs.shape:
            x = x + self.drop_path(inputs)

        return x

    def _make_divisible(self, out_channels, groups):
        ratio = out_channels // groups
        return int(groups * ratio)

    def _initialize_weights(self):
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

In [ ]:

import torch
import torch.nn as nn

class SABlock(nn.Module):

    def __init__(self, kernel_size=7):
        super().__init__()

        self.kernel_size = kernel_size

        self.conv2d = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        avg_pool = torch.mean(inputs, dim=1, keepdim=True)
        max_pool, _ = torch.max(inputs, dim=1, keepdim=True)

        x = torch.cat([avg_pool, max_pool], dim=1)
        x = self.conv2d(x)
        x = self.sigmoid(x)

        return x * inputs

class LCABlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()

        self.depthwise = nn.Conv2d(
            in_channels=in_channels,
            out_channels=in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            groups=in_channels,
            bias=False
        )

        self.gamma = nn.Parameter(torch.zeros(1, 1, 1, in_channels))
        self.beta = nn.Parameter(torch.zeros(1, 1, 1, in_channels))

    def forward(self, x):
        x = self.depthwise(x)
        x = x.permute(0, 2, 3, 1)
        group_norm_x = torch.norm(x, p=2, dim=(1, 2), keepdim=True)
        relative_important_x = group_norm_x / (group_norm_x.mean(dim=-1, keepdim=True) + 1e-6)
        x = self.gamma * (x * relative_important_x) + self.beta + x
        x = x.permute(0, 3, 1, 2)

        return x

class CABlock(nn.Module):

    def __init__(self, in_planes, ratio=16):
        super().__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.block = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.block(self.avg_pool(x))
        max_out = self.block(self.max_pool(x))
        out = avg_out + max_out

        return self.sigmoid(out) * x

# BACKBONE

In [ ]:
import timm
import torchvision
import torch
from torchvision.models.feature_extraction import create_feature_extractor
import torch
import torch.nn as nn

class BackboneFactory:
    @staticmethod
    def create_model(name: str):
        if name == "mobile_13th_block":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            return_nodes = {
                "features.13": "expand"
            }

            return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "mobile_10th_block":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            return_nodes = {
                "features.10": "expand"
            }

            return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "mobile_6th_block":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            return_nodes = {
                "features.6": "expand"
            }

            return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "mobile_expand_relu":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            return_nodes = {
                    "features.14.conv.0": "expand"
            }

            return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "ghostnet":
            backbone = timm.create_model('ghostnet_100', pretrained=True)
            backbone.classifier = nn.Sequential(
                nn.Linear(1280, 1),
            )

            return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "mobilenet":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            backbone.classifier[1] = nn.Sequential(
                nn.Linear(1280, 1),
            )

            return create_feature_extractor(backbone, return_nodes=return_nodes)
        elif name == "mobile_v2_cls":
            backbone = torchvision.models.mobilenet_v2(pretrained=True)
            backbone.classifier[1] = nn.Sequential(
                nn.Linear(1280, 4),
            )
            return backbone
            # return create_feature_extractor(backbone, return_nodes=return_nodes)

        # elif name == "shufflenet":
        #     backbone = torchvision.models.shufflenet_v2_x1_0(pretrained=True)
        #     backbone.fc = nn.Sequential(
        #         nn.Linear(1024, 1),
        #     )

        #     return create_feature_extractor(backbone, return_nodes=return_nodes)

        elif name == "mobilev3":
            backbone = torchvision.models.mobilenet_v3_small(pretrained=True)
            backbone.classifier[3] = nn.Sequential(
                nn.Linear(1024, 1),
            )
            return backbone

        else:
            raise ValueError(f"There is no model whose name is {name}.")


## GHOST MODULE

In [ ]:
import torch
import torch.nn as nn

class SeperableGhostModule(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        bias=False
    ):
        super().__init__()


        self.primary = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=in_channels,
                kernel_size=3,
                stride=1,
                padding=1,
                groups=in_channels,
                bias=bias
            ),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels//2,
                kernel_size=1,
                stride=1,
                bias=bias
            ),
            nn.BatchNorm2d(out_channels//2),
            nn.ReLU()
        )

        self.cheap = nn.Sequential(
            nn.Conv2d(
                in_channels=out_channels//2,
                out_channels=out_channels//2,
                kernel_size=kernel_size,
                stride=stride,
                padding=kernel_size//2,
                groups=out_channels//2,
                bias=bias
            ),
            nn.BatchNorm2d(out_channels//2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.primary(x)
        x_cheap = self.cheap(x)
        x = torch.concat([x, x_cheap], axis=1)
        return x

class GhostModule(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        bias=False
    ):
        super().__init__()

        self.primary = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels//2,
                kernel_size=1,
                stride=1,
                bias=bias
            ),
            nn.BatchNorm2d(out_channels//2),
            nn.ReLU()
        )

        self.cheap = nn.Sequential(
            nn.Conv2d(
                in_channels=out_channels//2,
                out_channels=out_channels//2,
                kernel_size=kernel_size,
                stride=stride,
                padding=kernel_size//2,
                groups=out_channels//2,
                bias=bias
            ),
            nn.BatchNorm2d(out_channels//2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.primary(x)
        x_cheap = self.cheap(x)
        x = torch.concat([x, x_cheap], axis=1)
        return x

In [ ]:
class SeperableConv2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        bias=False
    ):
        super().__init__()
        pass


In [ ]:
class ActivationFunction(nn.Module):
    def __init__(self):
        super().__init__()
        self.name = self.__class__.__name__
        self.config = {"name": self.name}
class ReLU_04(ActivationFunction):
    def forward(self, x):
        x = x * (x > 0).float()
        # * (x > 0) if true, return 1, therefore x * 1 = x, x <= 0 true then x > 0 = 1 then it is 0
        # If x >=4 then (x >= 4) = 1 and (x < 4) = 0; then it choose one of x or 4
        return ((x >= 4) * 4 + (x < 4) * x).float()

In [ ]:
from scipy.stats import spearmanr, pearsonr
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
# from pl_trainer.shuffle_trainer import MobileNet, QualityNet, InferenceModel

class DMGNet(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.backbone = BackboneFactory.create_model("mobile_torchvision_13")
        # self.backbone = net
        # for param in self.backbone.parameters():
        #     param.require_grad = False

#         for module in self.backbone.modules():
#             if isinstance(module, nn.BatchNorm2d):
#                 if hasattr(module, 'weight'):
#                     module.weight.requires_grad_(False)
#                 if hasattr(module, 'bias'):
#                     module.bias.requires_grad_(False)
# #                 module.affine = False
#                 module.eval()

#         self.init_conv = nn.Sequential(
#             nn.Conv2d(
#                 in_channels=576,
#                 out_channels=64,
#                 kernel_size=1,
#                 stride=1,
#                 bias=False
#             ),
#             nn.BatchNorm2d(64)
#         )

#         self.dpd_blocks = nn.Sequential(
#             DPDBlockV3(64, 384, 96, False),
#             DPDBlockV3(96, 576, 160, True),
#             DPDBlockV3(160, 960, 160, False),
#             DPDBlockV3(160, 960, 320, False)
#         )

        self.init_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=576,
                out_channels=32,
                kernel_size=1,
                stride=1,
                bias=False
            ),
            nn.BatchNorm2d(32)
        )

        self.dpd_blocks = nn.Sequential(
            DPDBlockV3(32, 192, 48, False),
            DPDBlockV3(48, 288, 64, True),
            DPDBlockV3(64, 384, 64, False),
            DPDBlockV3(64, 386, 96, False)
        )

#         self.smooth_conv = nn.Sequential(
#             nn.Conv2d(
#                 in_channels=320,
#                 out_channels=256,
#                 kernel_size=1,
#                 stride=1,
#                 bias=False
#             ),
#             nn.BatchNorm2d(256),
#             nn.ReLU()
#         )

        self.smooth_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=96,
                out_channels=144,
                kernel_size=1,
                stride=1,
                bias=False
            ),
            nn.BatchNorm2d(144),
            nn.ReLU()
        )

        self.gap = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )

        self.classifier = nn.Sequential(
            nn.Linear(144, 1),
            nn.ReLU()
        )

        self.loss_fn = torch.nn.MSELoss()
        self.test_metric = torch.nn.L1Loss()
        self.preds_epoch = []
        self.labels_epoch = []
        self.val_preds_epoch = []
        self.val_labels_epoch = []
        # self.test_preds_epoch = []
        # self.test_labels_epoch = []

    def forward(self, x):
        x = self.backbone(x)
        x = x["expand"]
        x = self.init_conv(x)
        x = self.dpd_blocks(x)
        x = self.smooth_conv(x)
        x = self.gap(x)
        x = self.classifier(x)

        return x

    def get_top_layers(self):
        return nn.Sequential(
            self.dpd_blocks,
            self.smooth_conv,
            self.gap,
            self.classifier
        )

#     def get_backbone(self):
#         return nn.Sequential(
#             self.backbone,
#             self.init_conv
#         )
    def get_backbone_feature(self, x):
        self.eval()

        with torch.no_grad():
            x = self.backbone(x)
            x = x["expand"]
            x = self.init_conv(x)

        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=20, min_lr=1e-7, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_srocc"}

    def _calculate_loss(self, batch):
        image, rating = batch
        y_pred = self.forward(image)
        y_pred = y_pred[:, 0]

        loss = self.loss_fn(y_pred, rating)
        self.preds_epoch = np.append(self.preds_epoch, y_pred.cpu().detach().numpy())
        self.labels_epoch = np.append(self.labels_epoch, rating.cpu().detach().numpy())
        rho_s, _ = spearmanr(np.squeeze(self.preds_epoch), np.squeeze(self.labels_epoch))
        rho_p, _ = pearsonr(np.squeeze(self.preds_epoch), np.squeeze(self.labels_epoch))
        # print(rho_s)
        # self.log("train_srocc", rho_s)
        # self.log("train_plcc", rho_p)

        return loss, rho_s, rho_p

    def _calculate_test_loss(self, batch):
        image, rating = batch
        y_pred = self.forward(image)
        y_pred = y_pred[:, 0]

        loss = self.test_metric(y_pred, rating)
        self.val_preds_epoch = np.append(self.val_preds_epoch, y_pred.cpu().detach().numpy())
        self.val_labels_epoch = np.append(self.val_labels_epoch, rating.cpu().detach().numpy())
        rho_s, _ = spearmanr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        rho_p, _ = pearsonr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))

        return loss, rho_s, rho_p

    def training_step(self, batch, batch_index):
        loss, rho_s, rho_p = self._calculate_loss(batch)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_srocc", rho_s, prog_bar=True)
        self.log("train_plcc", rho_p, prog_bar=True)
        return {"loss": loss, "srocc": rho_s, "plcc": rho_p}

    def on_train_epoch_end(self):
        rho_s, _ = spearmanr(np.squeeze(self.preds_epoch), np.squeeze(self.labels_epoch))
        rho_p, _ = pearsonr(np.squeeze(self.preds_epoch), np.squeeze(self.labels_epoch))
        self.log("train_srocc_epoch", rho_s)
        self.log("train_plcc_epoch", rho_p)
        self.preds_epoch = []
        self.labels_epoch = []
        print(f"Train SROCC = {rho_s} and Train PLCC = {rho_p}")

    def validation_step(self, batch, batch_index):
        loss, rho_s, rho_p = self._calculate_test_loss(batch)
        self.log("val_loss", loss)
        self.log("val_srocc", rho_s)
        self.log("val_plcc", rho_p)
        return {"loss": loss, "srocc": rho_s, "plcc": rho_p}

    def on_validation_epoch_end(self):
        rho_s, _ = spearmanr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        rho_p, _ = pearsonr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        self.log("val_srocc_epoch", rho_s)
        self.log("val_plcc_epoch", rho_p)
        self.val_preds_epoch = []
        self.val_labels_epoch = []
        print(f"Train SROCC = {rho_s} and Train PLCC = {rho_p}")

    def on_test_epoch_end(self):
        rho_s, _ = spearmanr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        rho_p, _ = pearsonr(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        mse = mean_squared_error(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        mae = mean_absolute_error(np.squeeze(self.val_preds_epoch), np.squeeze(self.val_labels_epoch))
        self.log("test_srocc_epoch", rho_s)
        self.log("test_plcc_epoch", rho_p)
        self.log("test_mse", mse)
        self.log("test_mae", mae)
        print(np.squeeze(self.val_preds_epoch))
        pd.DataFrame.from_dict({
            "preds":  np.squeeze(self.val_preds_epoch),
            "targets": np.squeeze(self.val_labels_epoch)
        }).to_csv("pred_score.csv", index=False)

        self.val_preds_epoch = []
        self.val_labels_epoch = []
        print(f"Test SROCC = {rho_s}, test PLCC = {rho_p}, test MSE = {mse} and test MAE = {mae}")


    def test_step(self, batch, batch_index):
        loss, rho_s, rho_p = self._calculate_test_loss(batch)

        self.log("test_loss", loss)
        self.log("test_srocc", rho_s)
        self.log("test_plcc", rho_p)
        return loss



In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchvision
from torchmetrics import F1Score, Accuracy

class InferenceModel(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.model = model
        self.focal_loss = torch.nn.CrossEntropyLoss(torch.tensor([50., 9., 3., 1.88, 50.]).to("cuda:1"))

    def forward(self, x):
        output = self.model(x)
        return output

class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.backbone = BackboneFactory.create_model("mobile_v2_cls")


    def forward(self, x):
        x = self.backbone(x)
        return x

class QualityNet(pl.LightningModule):
    def __init__(self, model=MobileNet()):
        super().__init__()

        self.model = model

        self.focal_loss = torch.nn.CrossEntropyLoss(torch.tensor([50., 9., 3., 1.88, 50.]).to("cuda:1"))
        self.f1_train = F1Score(task="multiclass", num_classes=5, average="macro")
        self.f1_val = F1Score(task="multiclass", num_classes=5, average="macro")
        self.acc_train = Accuracy(task="multiclass", num_classes=5)
        self.acc_val = Accuracy(task="multiclass", num_classes=5)
        self.validation_step_outputs = []

    def forward(self, x, label):
        x = self.model(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-5)
        # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=7, min_lr=1e-7, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_acc"}

    def _calculate_train_acc(self, batch):
        images, labels = batch

        logits = self.forward(images, labels)
        preds = torch.argmax(logits, dim=-1)

        f1 = self.f1_train(preds, labels)
        acc = self.acc_train(labels, preds)

        return acc, f1

    def _calculate_val_acc(self, batch):
        images, labels = batch

        logits = self.forward(images, labels)
        preds = torch.argmax(logits, dim=-1)

        self.f1_val.update(preds, labels)
        self.acc_val.update(labels, preds)

    def _calculate_loss(self, batch):
        image, label = batch
        y_pred = self.forward(image, label)

        loss = self.focal_loss(y_pred, label)
        return loss

    def _calculate_test_loss(self, batch):
        image, label = batch
        y_pred = self.forward(image, label)

        loss = self.focal_loss(y_pred, label)
        return loss, y_pred

    def training_step(self, batch, batch_index):
        image, label = batch
        loss, y_pred = self._calculate_test_loss(batch)
        acc, f1 = self._calculate_train_acc(batch)

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_f1", f1, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return {"loss": loss, "predictions": y_pred, "labels": label}

    def on_train_epoch_end(self):
        # compute metrics
        train_accuracy = self.acc_train.compute()
        train_f1 = self.f1_train.compute()
        # log metrics
        self.log("epoch_train_accuracy", train_accuracy)
        self.log("epoch_train_f1", train_f1)
        # reset all metrics
        self.acc_train.reset()
        self.f1_train.reset()
        print(f"\ntraining accuracy: {train_accuracy:.4}, "\
        f"f1: {train_f1:.4}")

#         preds = []
#         labels = []

#         for output in training_step_outputs:
#             logits = output['predictions']
#             preds += torch.argmax(logits, dim=-1).cpu().numpy().tolist()
#             labels += output['labels'].cpu().numpy().tolist()

        # print(classification_report(labels, preds))

    def validation_step(self, batch, batch_index):
        image, label = batch
        loss, y_pred = self._calculate_test_loss(batch)
        self._calculate_val_acc(batch)

        self.log("val_loss", loss)

        return {"val_loss": loss, "predictions": y_pred, "labels": label}

    # Validation epoch end
    def on_validation_epoch_end(self):
        val_accuracy = self.acc_val.compute()
        val_f1 = self.f1_val.compute()
        # log metrics
        self.log("val_acc", val_accuracy)
        self.log("val_f1", val_f1)
        # reset all metrics
        self.acc_val.reset()
        self.f1_val.reset()
        print(f"\nValidation accuracy: {val_accuracy:.4} "\
        f"f1: {val_f1:.4}")

#         preds = []
#         labels = []

#         for output in self.validation_step_outputs:
#             logits = output['predictions']
#             preds += torch.argmax(logits, dim=-1).cpu().numpy().tolist()
#             labels += output['labels'].cpu().numpy().tolist()

#         print(classification_report(labels, preds))

    def test_step(self, batch, batch_index):
        loss = self._calculate_loss(batch)
        self._calculate_val_acc(batch)

        # self.log("test_loss", loss)

        return loss

    def on_test_epoch_end(self):
        val_accuracy = self.acc_val.compute()
        val_f1 = self.f1_val.compute()
        # log metrics
        self.log("val_acc", val_accuracy)
        self.log("val_f1", val_f1)
        # reset all metrics
        self.acc_val.reset()
        self.f1_val.reset()
        print(f"\nTest accuracy: {val_accuracy:.4} "\
        f"Test f1: {val_f1:.4}")

class ShuffleNet(pl.LightningModule):
    def __init__(self):
        super().__init__()

#         # ShuffleNet
#         self.backbone = BackboneFactory.create_model("shuffle")

#         GhostNet
#         self.backbone = BackboneFactory.create_model("ghost")
#         self.backbone.classifier = nn.Sequential(
#             nn.Linear(1280, 1)

#         )
        #MobileNetV3
        self.backbone = BackboneFactory.create_model("mobilev3")
#         MobileNetV2
        # self.backbone = BackboneFactory.create_model("mobile")

        # MobileDPD10
#         self.backbone = MobileDPD10()

        # MobileDPD13
#         self.backbone = MobileDPD13()
        # MobileDPDBig
#         self.backbone = MobileDPDBig()

        self.loss_fn = torch.nn.L1Loss()
        self.test_metric = torch.nn.L1Loss()

    def forward(self, x):
        x = self.backbone(x)
#         x = x["expand"]
#         x = self.init_conv(x)
#         x = self.dpd_blocks(x)
#         x = self.smooth_conv(x)
#         x = self.gap(x)
#         x = self.classifier(x)
#         x = torch.where(x > 4, 4, x)
#         x = torch.where(x < 0, 0, x)

        return x * 4

    def get_top_layers(self):
        return nn.Sequential(
            self.dpd_blocks,
            self.smooth_conv,
            self.gap,
            self.classifier
        )

#     def get_backbone(self):
#         return nn.Sequential(
#             self.backbone,
#             self.init_conv
#         )
    def get_backbone_feature(self, x):
        self.eval()

        with torch.no_grad():
            x = self.backbone(x)
            x = x["expand"]
            x = self.init_conv(x)

        return x


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=7, min_lr=1e-7, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def _calculate_loss(self, batch):
        image, rating = batch
        y_pred = self.forward(image)
        y_pred = y_pred[:, 0]

        loss = self.loss_fn(y_pred, rating)
        return loss

    def _calculate_test_loss(self, batch):
        image, rating = batch
        y_pred = self.forward(image)
        y_pred = y_pred[:, 0]

        loss = self.test_metric(y_pred, rating)
        return loss

    def training_step(self, batch, batch_index):
        loss = self._calculate_loss(batch)
        self.log("train_loss", loss, prog_bar=True)

        return {"loss": loss}

    def validation_step(self, batch, batch_index):
        loss = self._calculate_test_loss(batch)
        self.log("val_loss", loss)

        return {"val_loss": loss}

    def test_step(self, batch, batch_index):
        loss = self._calculate_test_loss(batch)

        self.log("test_loss", loss)

        return loss



In [ ]:
import pytorch_lightning as pl
import os
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import torch
from torch.utils.data import DataLoader
from torchvision.models.feature_extraction import create_feature_extractor

parent_dir = "/data/image_review/data/koniq10/1024x768"
train_csv_path = "/data/image_review/data/koniq10/train_koniq.csv"
val_csv_path = "/data/image_review/data/koniq10/val_koniq.csv"
test_csv_path = "/dataimage_review/data/koniq10/test_koniq.csv"

quality_datamodule = QualityDataModule(parent_dir, train_csv_path, val_csv_path, batch_size=16)
# wandb_logger = WandbLogger(name="mobile_dpd_512x384", project="koniq10", log_model="all")
# wandb_logger.experiment.config["batch_size"] = 16

# net = InferenceModel(MobileNet())
# checkpoint_path = "weights/weight_koniq/mobile_koniq10_reduce_cls_epoch=32_val_f1=0.5576_val_acc=0.7440.ckpt"
# checkpoint_dict = torch.load(checkpoint_path, map_location="cuda:1")["state_dict"].keys()
# # print(checkpoint_dict)
# # del checkpoint_dict["focal_loss.weight"]
# net.load_state_dict(torch.load(checkpoint_path, map_location="cuda:1")["state_dict"])
# net = net.model.backbone
# return_nodes = {
#     "features.14.conv.0": "expand"
# }
# net = create_feature_extractor(net, return_nodes=return_nodes)
# print(net.model.backbone)
model = DMGNet.load_from_checkpoint(
    "weights/weight_koniq/mobiledpd_koniq10_reduce_epoch=58_val_srocc=0.9055_val_plcc=0.9198.ckpt")

# checkpoint_callback = ModelCheckpoint(
#     dirpath="weights/weight_koniq",
#     filename='mobiledpd_koniq10_reduce_{epoch}_{val_srocc:.4f}_{val_plcc:.4f}',
#     save_top_k=1,
#     verbose=True,
#     monitor="val_srocc",
#     mode="max",
# )

# trainer = pl.Trainer(accelerator="gpu",
#                      logger=wandb_logger,
#                      devices=[1],
#                      max_epochs=1000,
#                      callbacks=[checkpoint_callback])
# #  accumulate_grad_batches=1

trainer = pl.Trainer(accelerator="gpu",
                     devices=[1])

# # print("Training")
# trainer.fit(model, quality_datamodule)
test_data = QualityDataset(parent_dir, test_csv_path)
trainer.test(model, dataloaders=DataLoader(test_data, batch_size=32, num_workers=12))

# wandb.finish()



